In [1]:
import scipy
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from src.IterativeFitting import IterativeFitting as IF
from src.CorrFuncs import covariance_matrix, trend_est

In the Smith example, $p$ and $z$ are calculated from the initial frequencies of cases and controls. So maybe just use the crude estimates for $p$ and $z$.

Do we construct estimates for vx and for Lx everytime? I don't think so if we look at step 3. It seems like we input them from the information you give the algorithm and information that we get from study reports.

In [2]:
x = np.array([2,6,11])
Nx = np.array([337,167,186,212])
M1x = 451
Lx = np.array([np.log(0.80),np.log(1.16),np.log(1.57)])
vx = np.array([0.0542,0.0563,0.0563])
a0 = 165
A0x = np.array([74,90,122])
b0 = 172
B0x = np.array([93,96,90])
B0x_sum = np.sum(B0x) + b0
# vx = 1/a0 + 1/b0 + 1/A0x + 1/B0x

In [3]:
vx

array([0.0542, 0.0563, 0.0563])

Defining initial parameters of $p$ and $z$.

In [4]:
p0 = b0 / B0x_sum
z0 = B0x_sum / M1x

In [5]:
p0

0.38137472283813745

In [6]:
z0

1.0

Defining a function that allows us to construct confidence intervals

Get sum of squared differences and then check to see how microsoft solver works... Also look at R code probably, it will be easier. This is just an optimization problem for the following function. We need to find b0 and a0 that optimize the function. Maybe cvxpy?

According to the R code, we want to use scipy minimize. a0 and b0 will be the parameters we optimize over. Then the function should be longer but contain something like sum_sq_diff. 

In [ ]:
f_LogLik <- function (C_Val)  {
        A0 = C_Val[1]
        B0 = C_Val[2]
        
        if (CCPR == 1 | CCPR == 3) {         # case-control or X-Sectional (diseases+exposures)
          Vextra = RR_Val$Var - 1/A0 - 1/B0  
          Est_A  <- (1 + (A0*RR_Val$RR)/B0)/Vextra  # estimate of Ai
          Est_B  <- (1 + B0/(RR_Val$RR*A0))/Vextra  # estimate of Bi
        } else if(mode2 == 0) {  # prospective, diseases
          Vextra = RR_Val$Var + 1/A0 + 1/B0   
          Est_A  <- (1 + (A0*RR_Val$RR)/B0)/Vextra  # estimate of Ai
          Est_B  <- (1 + B0/(RR_Val$RR*A0))/Vextra  # estimate of Bi
        } else     {              #   ! prospective, exposures
          Vextra = RR_Val$Var - 1/A0 + 1/B0   
          Est_A  <- (1 - (A0*RR_Val$RR)/B0)/Vextra  # estimate of Ai
          Est_B  <- (-1 + B0/(RR_Val$RR*A0))/Vextra  # estimate of Bi
        }
        SumA = A0 + sum(Est_A)
        SumB = B0 + sum(Est_B)
        
        P1 = B0/SumB   # new estimate of (P-P')/P
        F1 = (P-P1)/P
        Z1 = SumB/SumA # estimate of (Z-Z')/Z
        F2 = (Z-Z1)/Z
        RR_Vextra <<- Vextra
        RR_A <<- Est_A
        RR_B <<- Est_B
        return(F1*F1 + F2*F2)
      }

In [7]:
def f_LogLik(C_Val):
    a0x = C_Val[0]
    b0x = C_Val[1]
    # global vx, Lx

    Vextra = vx - 1/a0x - 1/b0x
    Est_A = (1 + (a0x/b0x)*np.exp(Lx))/(Vextra)
    Est_B = (1 + b0x/(a0x*np.exp(Lx)))/(Vextra)
    
    SumA = a0x + np.sum(Est_A)
    SumB = b0x + np.sum(Est_B)

    p1 = b0x / SumB
    F1 = ((p0-p1)/p0)**2
    z1 = SumB/SumA
    F2 = ((z0-z1)/z0)**2
    
    # vx = 1/a0x + 1/b0x + 1/Est_A + 1/Est_B
    # # print(np.log(Est_B))
    # Lx = np.log(Est_A) + np.log(b0x) - np.log(a0x) - np.log(Est_B)

    return F1 + F2

In [8]:
a0_b0_res = scipy.optimize.minimize(f_LogLik,[a0,b0])

In [9]:
a0_b0_res

      fun: 3.7254864679456837e-10
 hess_inv: array([[6144.89733454, 3909.97483207],
       [3909.97483207, 6591.94908009]])
      jac: array([-1.37371661e-07, -2.31259192e-07])
  message: 'Optimization terminated successfully.'
     nfev: 78
      nit: 15
     njev: 26
   status: 0
  success: True
        x: array([ 96.26992359, 103.79444345])

In [10]:
a0, b0 = a0_b0_res.x[0], a0_b0_res.x[1]

In [11]:
a0

96.26992358923333

In [12]:
b0

103.79444345026278

In [13]:
a0/b0

0.927505561849897

In [17]:
Vextra = vx - 1/a0 - 1/b0
A = (1 + (a0/b0)*np.exp(Lx))/(Vextra)
B = (1 + b0/(a0*np.exp(Lx)))/(Vextra)

In [18]:
A

array([50.96842138, 57.22200793, 67.70428642])

In [19]:
B

array([68.69018295, 53.1849287 , 46.49432962])

In [17]:
vx - 1/a0 - 1/b0

array([0.03638643, 0.03848643, 0.03848643])

In [18]:
Lx

array([-0.22314355,  0.14842001,  0.45107562])

In [21]:
s = np.sqrt(1/A + 1/B + 1/a0 + 1/b0)
r = ((1/np.outer(s,s))*(1/a0 + 1/b0))[np.triu_indices(3,k=1)]
c = r*np.sqrt((np.outer(vx,vx))[np.triu_indices(3,k=1)])
C1 = np.diag(vx)
triu_indices = np.triu_indices(3,k=1)
tril_indices = np.tril_indices(3,k=-1)
C2 = C1.copy()
C2[triu_indices] = c
C = C2.copy()
C[tril_indices] = c

In [22]:
C

array([[0.0542    , 0.02002189, 0.02002189],
       [0.02002189, 0.0563    , 0.02002189],
       [0.02002189, 0.02002189, 0.0563    ]])

In [23]:
Cinv = np.linalg.inv(C)
vb_star = 1/(np.dot(x,np.dot(Cinv,x)))
b_star = vb_star*(np.dot(x,np.dot(Cinv,Lx)))

In [24]:
b_star

0.04588480137880012

In [25]:
vb_star

0.000420818983411542